# H2: CNP + Редкость устройств и сегментные пороги

**H2 (Product):** Редкие/новые устройства и отсутствующая карта (CNP) ↑ риск → device binding и лимиты.

In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features, customer_velocity
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack, thr_at_precision
from src.rarity import train_freqs, apply_freqs
from src.segments import mask_super_risky

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df = load_transactions(TX); fx = load_fx(FX)
df = convert_to_usd(df, fx); df = unpack_last_hour_activity(df); df = add_basic_time_features(df); df = customer_velocity(df)

train, test = split_time_aware(df)
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])

dev_freq, ip_freq = train_freqs(train)
X_tr_r = apply_freqs(X_tr, dev_freq, ip_freq); X_te_r = apply_freqs(X_te, dev_freq, ip_freq)

pipe = Pipeline([('prep', build_preprocessor(X_tr_r)), ('clf', build_logreg())]).fit(X_tr_r, y_tr)
p = pipe.predict_proba(X_te_r)[:,1]

seg = mask_super_risky(test)
thr_seg, rec_seg, p_seg = thr_at_precision(y_te[seg], p[seg], 0.90) if seg.any() else (None,None,None)
thr_glb, rec_glb, p_glb = thr_at_precision(y_te[~seg], p[~seg], 0.90)
decision = np.where(seg, p >= (thr_seg or 1.0), p >= (thr_glb or 1.0)).astype(int)

{'eval': eval_pack(y_te, p),
 'segment_thr': {'thr_seg':thr_seg, 'rec_seg':rec_seg, 'p_seg':p_seg, 'thr_glb':thr_glb, 'rec_glb':rec_glb, 'p_glb':p_glb},
 'policy_metrics': {'Precision': precision_score(y_te, decision, zero_division=0),
                    'Recall': recall_score(y_te, decision, zero_division=0),
                    'TrafficShare': float(decision.mean())}}

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


{'eval': {'roc_auc': 0.5001122214983779,
  'pr_auc': 0.1980998640895631,
  'threshold_at_precision': 0.9,
  'thr_value': None,
  'recall_at_precision': None,
  'precision_achieved': None},
 'segment_thr': {'thr_seg': None,
  'rec_seg': None,
  'p_seg': None,
  'thr_glb': None,
  'rec_glb': None,
  'p_glb': None},
 'policy_metrics': {'Precision': 0.0, 'Recall': 0.0, 'TrafficShare': 0.0}}

### Выводы по H2 (CNP + редкость устройств)

**Факты**
- Качество модели на текущих фичах ≈ случайное: **ROC-AUC ~0.500**, **PR-AUC ~0.198** (на уровне доли fraud).
- Достичь **Precision ≥ 0.90** не удалось (порог не найден для сегмента и вне сегмента).
- Двухпороговая политика не сработала: `Precision=0`, `Recall=0`, `TrafficShare=0` (фоллбек порога привёл к нулевым алертам).

**Интерпретация причин**
1. **Сигнал редкости «схлопнулся» на тесте**: большинство `device_fingerprint`/`ip_address` оказались «новыми» (freq=0), либо почти все — редкие → модель не различает риск внутри сегмента.  
2. **Флаги сегмента (CNP, cross-border, high-risk)** могли быть **слабо вариативны** или перекошены по времени/странам → сегмент S слишком мал/неотличим.  
3. **Пре-процессинг**: агрессивный `min_frequency` в OHE и медианная импутация могли обесценить слабые сигналы.

**Решения (минимальные доработки, чтобы «зажёгся» сигнал)**
- Усилить признаки редкости:
  - `device_is_new = (device_freq == 0)`
  - `ip_is_new = (ip_freq == 0)`
  - `device_is_rare = (device_freq <= 3)`
  - `ip_is_rare = (ip_freq <= 3)`
  - `rarity_score = 1.0 / (1 + device_freq) + 1.0 / (1 + ip_freq)`  
  и **взаимодействия** с `is_card_present==False`, `is_outside_home_country==True`.
- Смягчить кодирование категорий: снизить `min_frequency` в `OneHotEncoder` до **10**.
- Проверить наполнение сегмента:
  - `test['is_card_present'].value_counts(normalize=True)`,
  - `test['is_outside_home_country'].mean()`,
  - `test['is_high_risk_vendor'].mean()`,
  - `seg.sum()` — если `seg` слишком мал, используйте более широкий сегмент (например, CNP ∪ cross-border).
- Как «страховочную» политику до обучения лучшей модели — применять **квантильные пороги по amount_usd** внутри сегментов (P95 для CNP, P90 для cross-border, P80 для CNP∩CB∩HRV).

Итог по H2 на текущем запуске: гипотеза не подтверждена. Для следующей итерации добавляем «жёсткие» признаки новизны/редкости + взаимодействия, расширяем сегмент (если он мал), смягчаем OHE. Параллельно держим квантильную политику как fallback до появления устойчивого модельного сигнала.